# IMDLIB

## 1. Rainfall

### 1.1 Dowonlaod Yearwise Rainfall Data

In [ ]:
import imdlib as imd

# Define time (year) and data directory
start_year = 2015
end_year = 2024
data_dir = "D:/IMD/RAINFALL/"

# Download daily data
imd.get_data('rain', start_year, end_year, fn_format='yearwise', file_dir=data_dir)

#it will download data for whole India , further have to clip for study area...


Downloading: rain for year 2015


### 1.2 clipping

In [23]:
import geopandas as gpd
import rioxarray
import imdlib as imd

#paths and parameters
data_dir   = "D:/IMD/RAINFALL/"    
shapefile  = "D:/IMD/KOSI_BOUNDARY/KOSI-BOUNDARY.shp"
start_year = 2015
end_year   = 2024

#open IMD daily rainfall data which is in the unit mm/day
data = imd.open_data('rain', start_year, end_year, 'yearwise', data_dir)
rain = data.get_xarray().rio.write_crs("EPSG:4326")  # assign it to WGS84 

# Clip to aoi
aoi = gpd.read_file(shapefile).to_crs("EPSG:4326")
rain_masked = rain.rio.clip(aoi.geometry, aoi.crs, drop=True)

#clip to boundary
bounds = aoi.total_bounds   # [minx, miny, maxx, maxy]
rain_clipped = rain.rio.clip_box(
    minx=bounds[0], miny=bounds[1],
    maxx=bounds[2], maxy=bounds[3]
)
# Set spatial dimensions (for compatibility with rioxarray )
lat = [c for c in rain_clipped.coords if "lat" in c.lower()][0]
lon = [c for c in rain_clipped.coords if "lon" in c.lower()][0]
rain_clipped = rain_clipped.rio.set_spatial_dims(x_dim=lon, y_dim=lat)

#check it finally
print(rain_clipped)


<xarray.Dataset> Size: 263kB
Dimensions:      (lat: 2, lon: 4, time: 3653)
Coordinates:
  * lat          (lat) float64 16B 29.5 29.75
  * lon          (lon) float64 32B 79.0 79.25 79.5 79.75
  * time         (time) datetime64[ns] 29kB 2015-01-01 2015-01-02 ... 2024-12-31
    spatial_ref  int32 4B 0
Data variables:
    rain         (time, lat, lon) float64 234kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
Attributes:
    Conventions:  CF-1.7
    title:        IMD gridded data
    source:       https://imdpune.gov.in/
    history:      2025-08-18 07:37:28.122625 Python
    references:   
    comment:      


### 1.3 Save clipped data as NetCDF (Rectengular)


In [5]:
out_nc = "D:/IMD/KOSI_RAINFALL_2015_2024.nc"
rain_clipped.to_netcdf(out_nc)
print(f"Saved to {out_nc}")


Saved to D:/IMD/KOSI_RAINFALL_2015_2024.nc


### 1.4 Save the daily data as CSV

In [17]:
#declare csv path
csv_path = "D:/IMD/KOSI_RAINFALL_DAILY_AVG_2015_2024.csv"

#average it 
rain_daily_mean = rain_masked.mean(dim=["lat", "lon"]).to_pandas()

#format index
rain_daily_mean.index = rain_daily_mean.index.strftime("%Y%m%d")

# save it with auto column headers
rain_daily_mean = rain_daily_mean.drop("spatial_ref", axis=1, errors="ignore")
rain_daily_mean.to_csv(csv_path, header=["Rainfall_in_mm"], index_label="Date")


print(f" Daily AOI mean rainfall saved: {csv_path}")


 Daily AOI mean rainfall saved: D:/IMD/KOSI_RAINFALL_DAILY_AVG_2015_2024.csv


### 1.5 Save average rainnfall as Tiff file 

In [ ]:
rain_pixelwise_mean = rain_masked.mean(dim="time").squeeze(drop=True) #for box [] change it to rain_clipped

out_tif = "D:/IMD/KOSI_RAINFALL_PIXELWISE_MEAN_2015_2024.tif"
rain_pixelwise_mean.rio.to_raster(out_tif, dtype="float32")
print(f"Saved pixelwise mean rainfall (2015–2024) to {out_tif}")


#rain_pixelwise_mean = rain_clipped.mean(dim="time").squeeze(drop=True) #Box
# out_nc = "D:/IMD/KOSI_RAINFALL_PIXELWISE_MEAN_2015_2024.nc"
# rain_pixelwise_mean.to_netcdf(out_nc)
# print(f"Saved pixelwise mean rainfall (2015–2024) to {out_nc}")


Saved pixelwise mean rainfall (2015–2024) to D:/IMD/KOSI_RAINFALL_PIXELWISE_MEAN_2015_2024.tif
